In [2]:
from dotenv import load_dotenv
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import BingGroundingTool

# Load environment variables from .env file
load_dotenv()

project_client = AIProjectClient(
    endpoint=os.getenv("ENDPOINT"),
    subscription_id=os.getenv("SUBSCRIPTION_ID"),
    resource_group_name=os.getenv("RESOURCE_GROUP_NAME"),
    project_name=os.getenv("PROJECT_NAME"),
    credential=DefaultAzureCredential()
)

bing_connection = project_client.connections.get(
    connection_name=os.getenv("BING_CONNECTION_NAME")
)
conn_id = bing_connection.id
print(conn_id)

# Initialize agent bing tool and add the connection id
bing = BingGroundingTool(connection_id=conn_id)

# Create agent with the bing tool and process assistant run
with project_client:
    agent = project_client.agents.create_agent(
        model="gpt-4o",
        name="bing_agent",
        instructions="You are a helpful assistant",
        tools=bing.definitions,
        headers={"x-ms-enable-preview": "true"}
    )
    print(f"Created agent, ID: {agent.id}")

    # Create thread for communication
    thread = project_client.agents.create_thread()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="How is the weather in Seattle today?",
    )
    print(f"Created message, ID: {message.id}")

    # Create and process agent run in thread with tools
    run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Delete the assistant when done
    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")

    # Fetch and log all messages
    messages = project_client.agents.list_messages(thread_id=thread.id)
    print(f"Messages: {messages}")


/subscriptions/1278a874-89fc-418c-b6b9-ac763b000415/resourceGroups/aiagents-demo/providers/Microsoft.MachineLearningServices/workspaces/ai-standard-project-hz7g/connections/binggrounding
Created agent, ID: asst_iXdVO3pbqyOCMBAme1yXmYF1
Created thread, ID: thread_yJmMqFZiIHUVPf6lL928JKRk
Created message, ID: msg_OPrybrBJ0YKsvSVaTGQYWv2s
Run finished with status: RunStatus.COMPLETED
Deleted agent
Messages: {'object': 'list', 'data': [{'id': 'msg_LRuEZpjW72yyrhnC3WbPlCuk', 'object': 'thread.message', 'created_at': 1732167496, 'assistant_id': 'asst_iXdVO3pbqyOCMBAme1yXmYF1', 'thread_id': 'thread_yJmMqFZiIHUVPf6lL928JKRk', 'run_id': 'run_9WzjmNRVrqIm6WAZhycIVxIB', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': 'Today in Seattle, the weather is expected to be rainy. The conditions include showers and less wind compared to recent days【5†source】. Expect temperatures around 50°F during the day and down to 45°F at night【13†source】.', 'annotations': [{'type': 'url_citation', 